<a href="https://colab.research.google.com/github/sdias22/Processamento-Digital-de-Img/blob/main/pratica02_turma_a1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Carregando uma base de dados:
* Dados imobiliários: https://drive.google.com/file/d/1DVaPIcCzupbIY3U9yokWH0mnaTBRh2Ab/view?usp=sharing
* Template para carregar arquivo compartilhado: https://drive.google.com/u/3/uc?id=&export=download

In [ ]:
import numpy as np
import pandas as pd
#1DVaPIcCzupbIY3U9yokWH0mnaTBRh2Ab
df = pd.read_csv('https://drive.google.com/u/3/uc?id=1DVaPIcCzupbIY3U9yokWH0mnaTBRh2Ab&export=download')

Observando a base com shape, head e describe:

In [ ]:
df.shape
df.columns

Index(['Suburb', 'Address', 'Rooms', 'Type', 'Price', 'Method', 'SellerG',
       'Date', 'Distance', 'Postcode', 'Bedroom2', 'Bathroom', 'Car',
       'Landsize', 'BuildingArea', 'YearBuilt', 'CouncilArea', 'Lattitude',
       'Longtitude', 'Regionname', 'Propertycount'],
      dtype='object')

Escolhendo as colunas e separando características descritivas da característica alvo:

In [ ]:
sel_columns = ['Rooms', 'Type', 'Bathroom', 'Car', 'Landsize', 'Lattitude', 'Longtitude']
X = df[sel_columns]
y = df['Price']

In [ ]:
y.shape

(13580,)

Tratar atributos categóricos:

In [ ]:
X.head()
X.Type.value_counts()
type_dummies = pd.get_dummies(X.Type, drop_first = True)
type_dummies.head()
X = X.drop('Type', axis = 1)
X = X.assign(**type_dummies)
X.head()

,Rooms,Bathroom,Car,Landsize,Lattitude,Longtitude,t,u
0,2,1.0,1.0,202.0,-37.7996,144.9984,0,0
1,2,1.0,0.0,156.0,-37.8079,144.9934,0,0
2,3,2.0,0.0,134.0,-37.8093,144.9944,0,0
3,3,2.0,1.0,94.0,-37.7969,144.9969,0,0
4,4,1.0,2.0,120.0,-37.8072,144.9941,0,0


Verificando valores ausentes:

In [ ]:
X.isnull().sum()

Rooms          0
Bathroom       0
Car           62
Landsize       0
Lattitude      0
Longtitude     0
t              0
u              0
dtype: int64

Antes de aplicar o pré-processamento, temos que pensar na estratégia de teste, pois o pré-processamento deve ser ajustado apenas na parte de treinamento da base de dados.

Começando pela estratégia mais simples: separar uma parte dos dados para teste:

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y)

Ajustando pré-processamento aos dados de treinamento e aplicando nos dados de treinamento e teste:

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)
X_train[:5,:]

from sklearn.impute import KNNImputer
imputer = KNNImputer(n_neighbors = 3)
X_train = imputer.fit_transform(X_train)
X_test = imputer.transform(X_test)

Criando um modelo de classificação utilizando Knn:

In [ ]:
from sklearn import neighbors
knn = neighbors.KNeighborsRegressor(n_neighbors = 3)
knn.fit(X_train, y_train)
predicoes = (knn.predict(X_test))

In [ ]:
print(predicoes[:5])
print(y_test[:5])

[770833.33333333 632833.33333333 782333.33333333 663000.
 879000.        ]
6810     805000.0
9714     602000.0
12070    821000.0
7906     901000.0
1327     800000.0
Name: Price, dtype: float64


Ajustando o modelo aos dados de treinamento e testando nos dados de teste:

Dando uma olhada nos rótulos preditos e rótulos reais:

Usando uma métrica de avaliação, por exemplo, Mean Absolute Error: $MAE = \frac{\sum_{i=1}^{n}|y_i-p_i|}{n}$

Separar a base de dados uma vez em treinamento e teste pode não refletir a real qualidade do modelo.

Utilizando a técnica *cross-validation* podemos obter uma medida de avaliação baseada no treinamento e teste com toda a base de dados, da seguinte maneira (ilustração com 4 *folds*, ou seja, dividindo a base de dados em 4 partes):

[---------Base de dados----------]

[**teste** ][treino][treino][treino] (Experimento 1)

[treino][**teste** ][treino][treino] (Experimento 2)

[treino][treino][**teste** ][treino] (Experimento 3)

[treino][treino][treino][**teste** ] (Experimento 4)

A medida de avaliação final é dada como a média das medidas obtidas em todos os experimentos.

A ferramenta cross_val_score pode separar a base de dados e realizar um experimento com cada combinação de *folds* de treino e teste. Porém, é interessante criar os *folds* antes e passa-los como parâmetro para o cross_val_score. Isso permitirá que usemos a mesma divisão da base de dados em experimentos diferentes, tornando a comparação mais justa e confiável.

Criando os *folds*:

Como o cross-val-score vai executar o experimento para as combinações de treinamento e teste das partes da base de dados e o pré-processamento deve ser ajustado na parte de treinamento e aplicado em ambas as partes, podemos criar uma sequência de procedimentos *pipeline* para ser aplicada em cada experimento:

Rodando o *cross-validation* com o *pipeline* criado acima:

Observando os scores, sua média e desvio-padrão:

Criando uma grade de parâmetros a serem testados:

# Atividade
* Verificar outros métodos de imputação de valores faltantes:
 * retirar
 * média
 * adicionar coluna indicando se é valor real ou imputado
* Outros números de vizinhos: 7, 9...
* Árvore de decisão
 * from sklearn.tree import DecisionTreeRegressor
 * pré-poda (max_depth, min_samples_leaf, min_samples_split, min_impurity_decrease)